# Задача: определить траекторию ракеты

Выполнил: студент группы 562КМ, Юсов Александр

## 1.Данные
1. Зависимость плотности от высоты
2. Зависимость скорости звука от высоты
3. $C_p(M)$
4. Начальная скорость и высота
5. Расход массы(топлива) от времени

## 2.Предположения и упрощения
1. Траектория двумерная
2. Апроксимация линейная
3. При параметрах, выходящих за пределы заданных, используем последнее значение

## 3. Метод численного решения
#### Метод Рунге-Кутты 4-го порядка

$$
\dfrac{d\vec{s}}{dt} = \vec{F} (t, \vec{s}) \\
$$

$$
\vec{k_1} = \vec{F} (t_i , \vec{s_i} )
$$


$$
\vec{k_2} = \vec{F} (t_i + \dfrac{h}{2}, \vec{s_i} + \dfrac{h}{2} \vec{k_1})
$$

$$
\vec{k_3} = \vec{F} (t_i + \dfrac{h}{2}, \vec{s_i} + \dfrac{h}{2} \vec{k_2})
$$

$$
\vec{k_4} = \vec{F} (t_i + h, \vec{s_i} + h \vec{k_3}) \\
$$

$$
\vec{s_{i+1}} = \vec{s_i} + \dfrac{h}{6} (\vec{k_1} + 2 \vec{k_2} + 2 \vec{k_3} + \vec{k_4} ) \\
$$

## 4. Физическая модель
$$ \vec{s} = \begin{bmatrix}
       x \\         
       z \\
       ux \\
       uz
     \end{bmatrix}
;
\qquad
\vec{F} = \begin{bmatrix}
    \vec{V}\\
    \vec{g} + \dfrac{\vec{e_v}}{m(t)} \left\{ (U-V) \dfrac{dm}{dt}-\dfrac{Cx(\vec{s})\rho (\vec{s}) V^2 S}{2} \right\} 
    \end{bmatrix}
\\
g = 9.8 \\
\vec{e_v} = \dfrac{\vec{V}}{|V|}
$$

## 5. Результаты моделирования
### Взлёт с земли с последующим включением двигателей
$ 
V_0 = \begin{bmatrix}
    5 \\
    1000
\end{bmatrix}
; \qquad
r_0 = \begin{bmatrix}
    0 \\
    0
\end{bmatrix}
; \qquad
d = 85 \text{мм.}; \qquad
U = 5000 \text{м\с - скорость струи газа}
$

1. Масса от времени m(t) ![масса от времени m(t)](img/moft.png "масса от времени m(t)")
2. Cx(M) ![Cx(M)](img/Cxofh.png "Cx(M)")
3. a(h) ![a(h) не загрузился](img/aofh.png "a(h)")
4. density(h) ![density(h)](img/densityofh.png "density(h)")
5. Траектория полёта z(x) 
![Траектория полёта z(x)](img/zofx.png "Траектория полёта z(x)")
6. Остальные показатели
![Остальные показатели](img/Moft.png "Остальные показатели")


## 6.Код программы
#### 1. Класс для интерполяции данных
```python
class value:
    """
    class of interpolated values
    """
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def get(self, xvalue):
        """
        return yvalue = yi + ki*(xvalue - xi)
        """
        xprev = self.x[0]
        if xvalue < xprev: return self.y[0]
        for i, xi in enumerate(self.x[1:]):
            i += 1 # due to x starts from 1 index
            if xvalue >= xprev and xvalue < xi:
                return self.y[i-1]+(self.y[i]-self.y[i-1])*(xvalue-xprev)/(xi-xprev)
            else:
                xprev = xi
        return self.y[-1]

    def diff(self, xvalue):
        xprev = self.x[0]
        if xvalue == self.x[0]:
            return (self.y[1]-self.y[0])/(self.x[1] - self.x[0])
        for i, xi in enumerate(self.x[1:]):
            if xvalue >= xprev and xvalue < xi:
                return (self.y[i]-self.y[i-1])/(xi - xprev)
            else:
                xprev = xi
        return 0
```

#### 2. Модуль для численного расчёта методом Рунге-Кутты
cmethods.py
```python
import numpy as np
class RK:
    def __init__(self, f, V0, dt, params):
        """
        f - function 
        V0 - velosity - vector
        dt - step
        r0 = conditions on start (x0, z0) - vector
        params = [g, S, M, dmdt, Cx, density]
        """
        self.f = f
        if type(V0) == 'numpy.ndarray':
            self.V = V0 
        else:
            self.V = np.array(V0)
        self.dt = dt
        self.t = 0
        self.params = params
        
    def coefs(self):
        k1 = self.f(self.t, self.V, *self.params)
        k2 = self.f(self.t+self.dt/2, self.V+self.dt*k1/2, *self.params)
        k3 = self.f(self.t+self.dt/2, self.V+self.dt*k2/2, *self.params)
        k4 = self.f(self.t+self.dt, self.V+self.dt*k3, *self.params)
        return self.V + self.dt*(k1+2*k2+2*k3+k4)/6
    
    def nextstep(self):
        """
        change i+1 step
        """
        self.V = self.coefs()
        self.t += self.dt
```

#### 3. Построение траекторий 
```python
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cmethods import RK
from interpolation import value


class writer:
    def __init__(self, file, rewrite=True, header=None):
        flag = 'w' if rewrite else 'a+'
        self.file = open(file, flag)
        if header is not None:
            self.file.write(','.join(header)+'\n')

    def close(self):
        self.file.close()

    def write(self, params):
        if type(params) == list:
            self.file.write(','.join(params)+'\n')
        else:
            self.file.write(params+'\n')

                            
a_value = np.array([338, 338, 330, 325, 320, 312, 298, 292, 295, 295, 292, 292, 292])
density_value = np.array([1.18, 1.18, 1, .8, .62, .55, .4, .38, .35, .19, .17, .15, .1]) # depends on h
h_value = np.array([0, 200, 2000, 4000, 6000, 8000, 10000, 12000, 14000, 16000, 18000, 20000, 22000])
Cx_value = np.array([0.15, .18, .2, .4, .48, .42, .38, .35, .3]) # depends on Mach
Mach_value = np.array([0, 0.5, 0.75, 1, 1.2, 1.5, 2, 2.5, 3])
# Cx_value = np.zeros_like(Mach_value)
M_value = np.linspace(9.2, 8, 2)
time_value = np.array([10, 20])
g = 9.8
d = 85e-3
Sq = np.pi*(d**2)/4
r0 = np.array([0, 0], dtype='float64')
dt = 0.01
V0 = np.array([5, 1000], dtype='float64')
U = 5000.0
a = value(h_value, a_value)
density = value(h_value, density_value)
Cx = value(Mach_value, Cx_value)
M = value(time_value, M_value)


def F(t, s, gravity, Sq, M, Cx, density, a, U=0):
    currentCx = Cx.get(np.linalg.norm(s[2:], 2) / a.get(s[1]))
    if not (isinstance(M, float) or isinstance(M, int)):
        dmdt = M.diff(t)
        m = M.get(t)
    else:
        dmdt = 0
        m = M
    currentDensity = density.get(s[1])
    g = np.array([0, -gravity]) # vector of gravity
    ev = s[2:] / np.linalg.norm(s[2:], 2) # unit vector of velosity
    Rf = -currentCx*currentDensity*(np.linalg.norm(s[2:], 2)**2)*Sq / 2 # resistance force
    Mf = (U-np.linalg.norm(s[2:], 2))*dmdt # force from mass change
    return np.concatenate((s[2:], g + ev*(Mf + Rf)/m))
        
        
maxTime = 250
currentTime = 0
st = np.concatenate((r0, V0))
tr1 = RK(F, st, dt, params=[g, Sq, M, Cx, density, a, U]) # the last param is U
writer1 = writer('data/data.csv',
                 header=['t', 'x', 'z', 'Vx', 'Vz', 'M', 'X/mg'])
currentMach = np.linalg.norm(st[2:], 2)/a.get(st[1])
currentCx = Cx.get(currentMach)
currentDensity = density.get(st[1])
currentV = np.linalg.norm(st[2:], 2)
currentM = M.get(currentTime) if type(M) == value else M
writer1.write(list(map(str, [currentTime, st[0], st[1], st[2], st[3],
              currentMach,
              -currentCx*currentDensity*(currentV**2)*Sq / (2*currentM*g)])))
while currentTime <= maxTime and st[1] >= 0:
    # print(currentTime)
    tr1.nextstep()
    st = tr1.V
    currentMach = np.linalg.norm(st[2:], 2)/a.get(st[1])
    currentCx = Cx.get(currentMach)
    currentDensity = density.get(st[1])
    currentV = np.linalg.norm(st[2:], 2)
    currentM = M.get(currentTime) if type(M) == value else M
    writer1.write(list(map(str, [currentTime, st[0], st[1], st[2], st[3],
              currentMach,
              -currentCx*currentDensity*(currentV**2)*Sq / (2*currentM*g)])))
    currentTime += dt
writer1.close()


data = pd.read_csv('data/data.csv')
def plotAx(ax, title, xlabel, ylabel, x, y, grid=True):
    ax.set_title(title)
    ax.grid(grid)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.plot(x, y)
    
def plotData(title, xlabel, ylabel, x, y, grid=True):
    plt.title(title)
    plt.grid(grid)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y)
    
    
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(18, 14))
columns = list(data)


for i, ax in enumerate(axes.flatten()):
    plotAx(ax, '{0}(t)'.format(columns[i+1]), columns[0],
            columns[i+1], data[columns[0]], data[columns[i+1]])

    
plotData('z(x)', 'x, meters', 'z, meters', data['x'], data['z'])
timesArr = np.linspace(0, 100, 100)
plt.ylim(6, 10)
plt.grid()
plt.xlabel('t, seconds')
plt.ylabel('m, kg')
Moft = [M.get(t) for t in timesArr]
plt.plot(timesArr, Moft)
```